In [3]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

Se detectan valores incorrectos como '?' y valores negativos. Estos valores se entienden como un dato NaN.

In [4]:
data_train = pd.read_csv('/Users/emart/Documents/GitHub/mdata_dp3/data/train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
columns_train = data_train.columns
for column in columns_train:
    """ Si el valor de la columna es '?' o menor a 0, se reemplaza por None. """
    data_train[column] = data_train[column].replace('?', None)
    data_train[column] = data_train[column].astype(float)
    data_train[column] = data_train[column].apply(lambda x: None if x < 0 else x)

data_train.head()

In [ ]:
data_test = pd.read_csv('test.csv')

columns_test = data_test.columns
for column in columns_test:
    """ Si el valor de la columna es '?' o menor a 0, se reemplaza por None. """
    data_test[column] = data_test[column].replace('?', None)
    data_test[column] = data_test[column].astype(float)
    data_test[column] = data_test[column].apply(lambda x: None if x < 0 else x)

data_test.head()

In [ ]:
data_train['age*chol'] = data_train['age'] * data_train['chol']
data_train['age*trestbps'] = data_train['age'] * data_train['trestbps']

data_test['age*chol'] = data_test['age'] * data_test['chol']
data_test['age*trestbps'] = data_test['age'] * data_test['trestbps']

data_train = data_train.drop(['chol', 'trestbps'], axis=1)
data_test = data_test.drop(['chol', 'trestbps'], axis=1)

#### 01. Completar valores NaN por KNN

In [ ]:
X = data_train.drop('label', axis=1)
y = data_train['label'] 

# Inicializar el imputador KNN
knn_imputer = KNNImputer(n_neighbors=10)
X_imputed = knn_imputer.fit_transform(X)

X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

data_train_imputed = pd.concat([X_imputed_df, y], axis=1)

data_train_imputed.head()

In [ ]:
X_test = data_test

X_test_imputed = knn_imputer.transform(X_test)

data_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)
data_test_imputed.head()

#### 02. Normalizar los valores con StandarScaler

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [ ]:
data_train_scaled = pd.DataFrame(X_scaled, columns=X.columns)
data_train_scaled = pd.concat([data_train_scaled, y], axis=1)

In [ ]:
data_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

#### 03. Modelo Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# GBM
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
X_train = data_train_scaled.drop('label', axis=1)
y_train = data_train_scaled['label']

# Inicializar el modelo de Bosques Aleatorios
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

In [ ]:
y_pred = rf_model.predict(data_test_scaled)

df = pd.DataFrame({'ID': range(len(y_pred)), 'label': y_pred})
df['label'] = df['label'].astype(int)
df.to_csv('result.csv', index=False)
